In [1]:
import warnings

warnings.filterwarnings("ignore", category=Warning)

import pickle

from kaggle_titanic.config import cfg
from ml_assemblr.main_components.data_pod import DataPod

In [2]:
with open(cfg.research_cache_path / "01_dp.pkl", "rb") as f:
    dp: DataPod = pickle.load(f)

In [3]:
dp.peek_main_df()

1309 rows × 21 columns


,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Embarked_infrequent_sklearn,Split_0,Split_1,Split_2
0,1,0.0,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,train,train,train
1,2,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,train,train,train
2,3,1.0,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,train,train,train
3,4,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,train,train,valid
4,5,0.0,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,train,train,train


In [4]:
import h2o
from h2o.automl import H2OAutoML

In [5]:
_ = h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.371-b11, mixed mode)
  Starting server from C:\Users\Titta\.conda\envs\titanic\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Titta\AppData\Local\Temp\tmptm201ikg
  JVM stdout: C:\Users\Titta\AppData\Local\Temp\tmptm201ikg\h2o_Titta_started_from_python.out
  JVM stderr: C:\Users\Titta\AppData\Local\Temp\tmptm201ikg\h2o_Titta_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Bangkok
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 15 days
H2O_cluster_name:,H2O_from_python_Titta_a9eems
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.027 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [6]:
feature_label_cols = dp.main_column_type.features + dp.main_column_type.labels

In [7]:
train_data = dp.slice_df("train", feature_label_cols, split_idx_in_column_type=1)
valid_data = dp.slice_df("valid", feature_label_cols, split_idx_in_column_type=1)

In [8]:
dp.slice_df("train", feature_label_cols, split_idx_in_column_type=1)["Survived"]

0      0.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
883    0.0
884    0.0
887    1.0
888    0.0
890    0.0
Name: Survived, Length: 712, dtype: float64

In [9]:
train_data["Survived"] = train_data["Survived"].astype("category")
valid_data["Survived"] = valid_data["Survived"].astype("category")

In [10]:
predictors = dp.main_column_type.features
response = dp.main_column_type.labels[0]

In [11]:
h2o_train_data = h2o.H2OFrame(train_data)
h2o_valid_data = h2o.H2OFrame(valid_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
h2o_train_data["Survived"] = h2o_train_data["Survived"].asfactor()
h2o_valid_data["Survived"] = h2o_valid_data["Survived"].asfactor()

In [13]:
aml = H2OAutoML(max_models=10, seed=42)
aml.train(x=predictors, y=response, training_frame=h2o_train_data)

AutoML progress: |
20:50:13.242: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),6/10
# GBM base models (used / total),3/6
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [14]:
lb = aml.leaderboard

In [15]:
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20240428_205013,0.861339,0.4222,0.840585,0.195374,0.362126,0.131135
StackedEnsemble_BestOfFamily_1_AutoML_1_20240428_205013,0.859839,0.42563,0.838342,0.195374,0.362919,0.13171
DRF_1_AutoML_1_20240428_205013,0.859058,0.492564,0.836789,0.20672,0.374748,0.140436
GBM_2_AutoML_1_20240428_205013,0.858696,0.430919,0.833134,0.194148,0.369054,0.136201
GBM_grid_1_AutoML_1_20240428_205013_model_1,0.858146,0.437299,0.829624,0.194601,0.372883,0.139041
GBM_5_AutoML_1_20240428_205013,0.85794,0.421021,0.838457,0.197207,0.36305,0.131805
XRT_1_AutoML_1_20240428_205013,0.853003,0.488402,0.803656,0.21156,0.393776,0.155059
GBM_4_AutoML_1_20240428_205013,0.852755,0.438624,0.828476,0.198938,0.372967,0.139105
GBM_3_AutoML_1_20240428_205013,0.849116,0.444014,0.811583,0.190904,0.373535,0.139528
GLM_1_AutoML_1_20240428_205013,0.843922,0.452066,0.81337,0.212065,0.378651,0.143376


In [16]:
predictions = aml.leader.predict(h2o_valid_data)
predictions

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.903426,0.0965744
1,0.387845,0.612155
0,0.790621,0.209379
0,0.956827,0.0431728
0,0.88424,0.11576
1,0.0888657,0.911134
0,0.834659,0.165341
1,0.514596,0.485404
1,0.429962,0.570038
1,0.591835,0.408165


In [17]:
valid_data.head(10)

,Sex,Age,SibSp,Parch,Fare,Cabin,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Embarked_infrequent_sklearn,Survived
5,male,NaN,0,0,8.4583,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
10,female,4.0,1,1,16.7000,G6,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
23,male,28.0,0,0,35.5000,A6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
25,female,38.0,1,5,31.3875,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
30,male,40.0,0,0,27.7208,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
31,female,NaN,1,0,146.5208,B78,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
33,male,66.0,0,0,10.5000,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
39,female,14.0,1,0,11.2417,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
44,female,19.0,0,0,7.8792,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
49,female,18.0,1,0,17.8000,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [18]:
h2o_valid_data

Sex,Age,SibSp,Parch,Fare,Cabin,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Embarked_infrequent_sklearn,Survived
male,nan,0,0,8.4583,nan,0,0,1,0,1,0,0,0
female,4,1,1,16.7,G6,0,0,1,0,0,1,0,1
male,28,0,0,35.5,A6,1,0,0,0,0,1,0,1
female,38,1,5,31.3875,nan,0,0,1,0,0,1,0,1
male,40,0,0,27.7208,nan,1,0,0,1,0,0,0,0
female,nan,1,0,146.521,B78,1,0,0,1,0,0,0,1
male,66,0,0,10.5,nan,0,1,0,0,0,1,0,0
female,14,1,0,11.2417,nan,0,0,1,1,0,0,0,1
female,19,0,0,7.8792,nan,0,0,1,0,1,0,0,1
female,18,1,0,17.8,nan,0,0,1,0,0,1,0,0


In [19]:
test_performance = aml.leader.model_performance(h2o_valid_data)

In [20]:
aml.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20240428_205013,0.861339,0.4222,0.840585,0.195374,0.362126,0.131135
StackedEnsemble_BestOfFamily_1_AutoML_1_20240428_205013,0.859839,0.42563,0.838342,0.195374,0.362919,0.13171
DRF_1_AutoML_1_20240428_205013,0.859058,0.492564,0.836789,0.20672,0.374748,0.140436
GBM_2_AutoML_1_20240428_205013,0.858696,0.430919,0.833134,0.194148,0.369054,0.136201
GBM_grid_1_AutoML_1_20240428_205013_model_1,0.858146,0.437299,0.829624,0.194601,0.372883,0.139041
GBM_5_AutoML_1_20240428_205013,0.85794,0.421021,0.838457,0.197207,0.36305,0.131805
XRT_1_AutoML_1_20240428_205013,0.853003,0.488402,0.803656,0.21156,0.393776,0.155059
GBM_4_AutoML_1_20240428_205013,0.852755,0.438624,0.828476,0.198938,0.372967,0.139105
GBM_3_AutoML_1_20240428_205013,0.849116,0.444014,0.811583,0.190904,0.373535,0.139528
GLM_1_AutoML_1_20240428_205013,0.843922,0.452066,0.81337,0.212065,0.378651,0.143376


In [21]:
leaderboard = aml.leaderboard
leaderboard_df = leaderboard.as_data_frame()
leaderboard_df["accuracy"] = [
    model.model_performance(h2o_test_data).accuracy() for model in aml.leaderboard
]

c:\Users\Titta\.conda\envs\titanic\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


AttributeError: 'H2OFrame' object has no attribute 'model_performance'

In [ ]:
accuracies = []
for model_id in leaderboard_df["model_id"]:
    model = h2o.get_model(model_id)
    performance = model.model_performance(h2o_valid_data)
    accuracy = performance.accuracy()
    accuracies.append(accuracy)

In [ ]:
performance.accuracy()

In [ ]:
performance

In [ ]:
performance.accuracy()

In [ ]:
leaderboard_df["accuracy"] = accuracies

In [ ]:
leaderboard_df